---

# Lib

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer

from preprocessing.clean import vn_text_clean
from preprocessing.tokenize import vn_word_tokenize
from preprocessing.remove_stopwords import remove_stopwords
from utils import parse_label, matrix_labels

[nltk_data] Downloading package punkt to C:\Users\DANG
[nltk_data]     CUONG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

# Read file

In [2]:
# Train
df_train = pd.read_csv("../data/raw/train.csv")
df_train.columns = df_train.columns.str.strip().str.lower().str.replace(' ', '_')

In [3]:
# Validation
df_val = pd.read_csv("../data/raw/val.csv")
df_val.columns = df_val.columns.str.strip().str.lower().str.replace(' ', '_')

In [4]:
df_train = df_train[["comment", "label"]].copy()
df_val = df_val[["comment", "label"]].copy()

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3593 entries, 0 to 3592
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  3593 non-null   object
 1   label    3593 non-null   object
dtypes: object(2)
memory usage: 56.3+ KB


In [6]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040 entries, 0 to 1039
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1040 non-null   object
 1   label    1040 non-null   object
dtypes: object(2)
memory usage: 16.4+ KB


---

# Clean text

In [7]:
df_train["comment"] = df_train["comment"].apply(lambda x: vn_text_clean(x))
df_val["comment"] = df_val["comment"].apply(lambda x: vn_text_clean(x))

In [8]:
df_train.iloc[0, 0]

'vf6 xe cho cá nhân dùng chứ gia đình dùng thì không ok mà việt nam chủ yếu mua xe dùng cho gia đình chứ ít cá nhân'

In [9]:
df_val.iloc[0,0]

'đại lý quá ít và dường như sắp đóng cửa là rào cảm khiến dân ta cũng còn em dè khi mua nissan chứ gắn logo toy hay honda thử xem là khác bọt liền'

---

# Tokenize

In [10]:
df_train["comment"] = df_train["comment"].apply(vn_word_tokenize, method="underthesea")
df_val["comment"] = df_val["comment"].apply(vn_word_tokenize, method="underthesea")

In [11]:
df_train.iloc[0, 0]

'vf6 xe cho cá_nhân dùng chứ gia_đình dùng thì không ok mà việt_nam chủ_yếu mua xe dùng cho gia_đình chứ ít cá_nhân'

In [12]:
df_val.iloc[0, 0]

'đại_lý quá ít và dường_như sắp đóng_cửa là rào_cảm khiến dân ta cũng còn em dè khi mua nissan chứ gắn logo toy hay honda thử xem là khác bọt liền'

---

# Remove stopwords

In [13]:
# Backup text for remove stopwords
df_train["text_backup"] = df_train["comment"].copy()
df_val["text_backup"] = df_val["comment"].copy()

In [16]:
def remove_stopwords_wrapper(df: pd.DataFrame) -> pd.DataFrame:
    # Remove stopwords with fallback if result becomes empty/NaN
    post = df["comment"].apply(remove_stopwords)

    # Normalize empties to NaN to enable fallback
    post = post.replace("", np.nan)

    print(post.isnull().sum())

    # Fallback to backup text where stopword removal produced NaN
    fallback = post.fillna(df["text_backup"])

    df["text"] = fallback
    
    return df

In [17]:
df_train = remove_stopwords_wrapper(df_train)
df_val = remove_stopwords_wrapper(df_val)

12
1


In [18]:
df_train.iloc[0, 0]

'vf6 xe cho cá_nhân dùng chứ gia_đình dùng thì không ok mà việt_nam chủ_yếu mua xe dùng cho gia_đình chứ ít cá_nhân'

In [19]:
df_val.iloc[0, 0]

'đại_lý quá ít và dường_như sắp đóng_cửa là rào_cảm khiến dân ta cũng còn em dè khi mua nissan chứ gắn logo toy hay honda thử xem là khác bọt liền'

In [20]:
# Remove backup columns
df_train.drop(columns="text_backup", axis=1, inplace=True)
df_val.drop(columns="text_backup", axis=1, inplace=True)

---

# Save file

In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3593 entries, 0 to 3592
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  3593 non-null   object
 1   label    3593 non-null   object
 2   text     3593 non-null   object
dtypes: object(3)
memory usage: 84.3+ KB


In [22]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040 entries, 0 to 1039
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1040 non-null   object
 1   label    1040 non-null   object
 2   text     1040 non-null   object
dtypes: object(3)
memory usage: 24.5+ KB


In [23]:
df_train.to_csv("../data/processed/train.csv", index=False)
df_val.to_csv("../data/processed/val.csv", index=False)